In [ ]:
#that would be sick if i could record the phone calls on my iphone device and then transcribe the call and add it to the database what in this case is the google sheet

In [5]:
import asyncio
from playwright.async_api import async_playwright

async def visit_autoscout24_dealers():
    async with async_playwright() as p:
        # Launch browser
        browser = await p.chromium.launch(headless=False)  # Set headless=True for production
        
        # Create new page
        page = await browser.new_page()
        
        # Navigate to the dealers page
        url = "https://www.autoscout24.de/haendler/#?city=Berlin"
        await page.goto(url)
        
        # Wait for the main content to load
        await page.wait_for_selector("h1", timeout=10000)  # Waits for the first h1 element

        # Wait for and click the cookie consent button
        accept_button = await page.wait_for_selector('button[data-testid="as24-cmp-accept-all-button"]')
        await accept_button.click()
        

        await page.wait_for_timeout(60000)  # Wait for 60 seconds
        # Close browser
        await browser.close()

# For Jupyter notebook, use this:
await visit_autoscout24_dealers()

# If running as a script, use this instead:
# if __name__ == "__main__":
#     asyncio.run(visit_autoscout24_dealers())